In [2]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("2c.csv", delimiter=";")
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], format="%d/%m/%Y %H:%M")

In [3]:
for i in ["TRIP", "TEMPERATURE", "FUEL", "PERSONS"]:
    df[i].fillna(method="ffill", inplace=True)

C:\Users\canut\AppData\Local\Temp\ipykernel_1096\1881467328.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[i].fillna(method='ffill', inplace=True)


In [6]:
df["ALTITUDE"].fillna(
    df["ALTITUDE"].rolling(window=15, min_periods=1).mean(), inplace=True
)


df["ALTITUDE"] = df["ALTITUDE"].round(0)

In [7]:
dn = list(df.select_dtypes(include=np.number).columns)

for i in dn:
    df[i] = df[i].astype(int)

In [8]:
df[df["ALTITUDE"].isna()].index

Index([], dtype='int64')

In [31]:
for i in [366, 367]:
    df.at[i, "ALTITUDE"] = 106

In [33]:
df.at[387, "ALTITUDE"] = 98

In [9]:
df.to_csv("test_clean_2.csv")

In [46]:
list_t = list(df[df["TIMESTAMP"].notnull()].index)

In [47]:
list_t

[0, 42, 43, 101, 102, 147, 148, 202, 203, 286, 344, 345, 405]

In [41]:
pd.date_range(
    start=pd.to_datetime(str(df["Timestamp"][0])),
    end=pd.to_datetime(str(df["Timestamp"][6])),
    periods=7,
)

DatetimeIndex(['1900-01-01 18:01:00', '1900-01-01 18:02:40',
               '1900-01-01 18:04:20', '1900-01-01 18:06:00',
               '1900-01-01 18:07:40', '1900-01-01 18:09:20',
               '1900-01-01 18:11:00'],
              dtype='datetime64[ns]', freq=None)

In [5]:
df = pd.read_csv("test_clean.csv", delimiter=",")
# df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], format='%d/%m/%Y %H:%M')

In [11]:
lr = df[df["TIMESTAMP"].notnull()].index
lr

Index([0, 42, 43, 101, 102, 147, 148, 202, 203, 286, 344, 345, 405], dtype='int64')

In [37]:
def get_even_lists(lst):
    even_pos = []
    odd_pos = []
    lim = len(lst)

    for i in range(len(lst)):
        if i % 2 == 1:
            odd_pos.append(lst[i])
        else:
            even_pos.append(lst[i])
    return odd_pos, even_pos

In [38]:
get_even_lists(lr)

([42, 101, 147, 202, 286, 345], [0, 43, 102, 148, 203, 344, 405])

In [14]:
for pd.date_range(
    start=pd.to_datetime(str(df["TIMESTAMP"][0])),
    end=pd.to_datetime(str(df["TIMESTAMP"][42])),
    periods=(42 - 0),
).round(freq="min")

DatetimeIndex(['2024-04-05 18:01:00', '2024-04-05 18:01:00',
               '2024-04-05 18:02:00', '2024-04-05 18:02:00',
               '2024-04-05 18:03:00', '2024-04-05 18:03:00',
               '2024-04-05 18:03:00', '2024-04-05 18:04:00',
               '2024-04-05 18:04:00', '2024-04-05 18:05:00',
               '2024-04-05 18:05:00', '2024-04-05 18:06:00',
               '2024-04-05 18:06:00', '2024-04-05 18:06:00',
               '2024-04-05 18:07:00', '2024-04-05 18:07:00',
               '2024-04-05 18:08:00', '2024-04-05 18:08:00',
               '2024-04-05 18:08:00', '2024-04-05 18:09:00',
               '2024-04-05 18:09:00', '2024-04-05 18:10:00',
               '2024-04-05 18:10:00', '2024-04-05 18:11:00',
               '2024-04-05 18:11:00', '2024-04-05 18:11:00',
               '2024-04-05 18:12:00', '2024-04-05 18:12:00',
               '2024-04-05 18:13:00', '2024-04-05 18:13:00',
               '2024-04-05 18:13:00', '2024-04-05 18:14:00',
               '2024-04-

In [ ]:
pd.date_range(
    start=pd.to_datetime(str(df["TIMESTAMP"][0])),
    end=pd.to_datetime(str(df["TIMESTAMP"][42])),
    periods=(42 - 0),
).round(freq="min")

DatetimeIndex(['2024-04-05 18:01:00', '2024-04-05 18:01:00',
               '2024-04-05 18:02:00', '2024-04-05 18:02:00',
               '2024-04-05 18:03:00', '2024-04-05 18:03:00',
               '2024-04-05 18:03:00', '2024-04-05 18:04:00',
               '2024-04-05 18:04:00', '2024-04-05 18:05:00',
               '2024-04-05 18:05:00', '2024-04-05 18:06:00',
               '2024-04-05 18:06:00', '2024-04-05 18:06:00',
               '2024-04-05 18:07:00', '2024-04-05 18:07:00',
               '2024-04-05 18:08:00', '2024-04-05 18:08:00',
               '2024-04-05 18:08:00', '2024-04-05 18:09:00',
               '2024-04-05 18:09:00', '2024-04-05 18:10:00',
               '2024-04-05 18:10:00', '2024-04-05 18:11:00',
               '2024-04-05 18:11:00', '2024-04-05 18:11:00',
               '2024-04-05 18:12:00', '2024-04-05 18:12:00',
               '2024-04-05 18:13:00', '2024-04-05 18:13:00',
               '2024-04-05 18:13:00', '2024-04-05 18:14:00',
               '2024-04-